# Introduction à la librairie PyTorch -- Solutions
Matériel de cours rédigé par Pascal Germain, 2019
************

### Partie 1

In [ ]:
class regression_logistique:
    def __init__(self, rho=.01, eta=0.4, nb_iter=50, seed=None):
        # Initialisation des paramètres de la descente en gradient
        self.rho = rho         # Paramètre de regularisation
        self.eta = eta         # Pas de gradient
        self.nb_iter = nb_iter # Nombre d'itérations
        self.seed = seed       # Germe du générateur de nombres aléatoires
        
        # Initialisation  trace de la descente de gradient
        self.w_list = list()   
        self.obj_list = list()
        
    def _trace(self, w, obj):
        self.w_list.append(np.array(w.detach()))
        self.obj_list.append(obj.item())      
        
    def apprentissage(self, x, y):
        if self.seed is not None:
            torch.manual_seed(self.seed)
        
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32) 

        n, d = x.shape
        self.w = torch.randn(d, requires_grad=True)
    
        for t in range(self.nb_iter + 1):
            xw = x @ self.w
            w2 = self.w @ self.w
            loss = torch.mean(- y * xw + torch.log(1 + torch.exp(xw))) + self.rho * w2 / 2          
            self._trace(self.w, loss)
  
            if t < self.nb_iter:
                loss.backward()
                with torch.no_grad():
                    self.w -= self.eta * self.w.grad
                
                self.w.grad.zero_()
                
    def prediction(self, x):
        x = torch.tensor(x, dtype=torch.float32)
        
        with torch.no_grad():
            pred = x @ self.w
                        
        return np.array(pred.numpy() > 0, dtype=np.int)

In [ ]:
class regression_logistique_avec_biais:
    def __init__(self, rho=.01, eta=0.4, nb_iter=50, seed=None):
        # Initialisation des paramètres de la descente en gradient
        self.rho = rho         # Paramètre de regularisation
        self.eta = eta         # Pas de gradient
        self.nb_iter = nb_iter # Nombre d'itérations
        self.seed = seed       # Germe du générateur de nombres aléatoires
        
        # Initialisation  trace de la descente de gradient
        self.w_list = list()   
        self.b_list = list()
        self.obj_list = list()
        
    def _trace(self, w, b, obj):
        self.w_list.append(np.array(w.detach()))
        self.b_list.append(b.item())
        self.obj_list.append(obj.item()) 
        
    def apprentissage(self, x, y):
        if self.seed is not None:
            torch.manual_seed(self.seed)
        
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32) 

        n, d = x.shape
        self.w = torch.randn(d, requires_grad=True)
        self.b = torch.zeros(1, requires_grad=True)
           
        for t in range(self.nb_iter + 1):
            xw = x @ self.w + self.b
            w2 = self.w @ self.w 
            loss = torch.mean(- y * xw + torch.log(1 + torch.exp(xw))) + self.rho * w2 / 2          
            self._trace(self.w, self.b, loss)
  
            if t < self.nb_iter:
                loss.backward()
                with torch.no_grad():
                    self.w -= self.eta * self.w.grad
                    self.b -= self.eta * self.b.grad
                    
                self.w.grad.zero_()
                self.b.grad.zero_()
                
    def prediction(self, x):
        x = torch.tensor(x, dtype=torch.float32)
        
        with torch.no_grad():
            pred = x @ self.w + self.b
            
        return np.array(pred.numpy() > 0, dtype=np.int)

### Partie 2

In [ ]:
class reseau_classification:
    def __init__(self, nb_neurones=4, eta=0.4, alpha=0.1, nb_iter=50, seed=None):
        # Architecture du réseau
        self.nb_neurones = nb_neurones # Nombre de neurones sur la couche cachée
        
        # Initialisation des paramètres de la descente en gradient
        self.eta = eta         # Pas de gradient
        self.alpha = alpha     # Momentum
        self.nb_iter = nb_iter # Nombre d'itérations
        self.seed = seed       # Germe du générateur de nombres aléatoires
        
        # Initialisation des listes enregistrant la trace de l'algorithme
        self.w_list = list()   
        self.obj_list = list()
        
    def _trace(self, obj):
        self.obj_list.append(obj.item())      
        
    def apprentissage(self, x, y):
        if self.seed is not None:
            torch.manual_seed(self.seed)
        
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

        n, d = x.shape
        self.model = nn.Sequential(
            torch.nn.Linear(d, self.nb_neurones),
            torch.nn.ReLU(),
            torch.nn.Linear(self.nb_neurones, 1),
            torch.nn.Sigmoid()
        )
        
        perte_logistique = nn.BCELoss()
        optimiseur = torch.optim.SGD(self.model.parameters(), lr=self.eta, momentum=self.alpha)
                   
        for t in range(self.nb_iter + 1):
            y_pred = self.model(x)
            perte = perte_logistique(y_pred, y)         
            self._trace(perte)
  
            if t < self.nb_iter:
                perte.backward()
                optimiseur.step()
                optimiseur.zero_grad()
                
    def prediction(self, x):
        x = torch.tensor(x, dtype=torch.float32)
        
        with torch.no_grad():
            pred = self.model(x)
            
        pred = pred.squeeze()
        return np.array(pred > .5, dtype=np.int)